In [1]:
# Standard Library Imports
import json
from pathlib import Path


# Third-Party Libraries
import numpy as np
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, models, transforms

# Avalanche: Continual Learning Framework
## Benchmarks
from avalanche.benchmarks.classic import SplitCIFAR100, SplitCIFAR10
from avalanche.benchmarks.datasets.torchvision_wrapper import CIFAR10
from avalanche.benchmarks.scenarios import CLExperience
from avalanche.benchmarks.utils.flat_data import ConstantSequence

## Models
from avalanche.models import (
    MultiHeadClassifier,
    MultiTaskModule,
    MTSimpleMLP,
    MTSimpleCNN,
    PNN,
)

## Training Strategies
from avalanche.training.supervised import Naive, EWC, LwF

## Plugins and Logging
from avalanche.logging import InteractiveLogger, TextLogger
from avalanche.training.plugins import EvaluationPlugin, LRSchedulerPlugin

## Evaluation Metrics
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics,
    loss_metrics,
    timing_metrics,
    cpu_usage_metrics,
    confusion_matrix_metrics,
    disk_usage_metrics,
)

import random

In [2]:
SAVE = False
import os

if SAVE:
    os.chdir('/home/uregina/DL_Project')
    print(os.getcwd())

# For saving the datasets/models/results/log files

if SAVE:
    DATASET_NAME = "SplitCIFAR100"
    ROOT = Path("/home/uregina/DL_Project")
    DATA_ROOT = ROOT / DATASET_NAME
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
seed = 0

DATASET_NAME = "SplitCIFAR100"
NUM_CLASSES = {
    "SplitCIFAR100": 100
}

# Define hyperparameters/scheduler/augmentation

SETTING = '1'# either '0' setting: 50 experiences (2 classes per experience) and 10 tasks or 
#'1' setting 10 experiences (10 classes per experience) and 5 tasks  corresponds to 10 and 4

if SETTING == '1':
    HPARAM = {
    "batch_size": 128,        
    "num_epoch": 10,           
    "start_lr": 0.01,
    "alpha": 1,   
    "temperature": 2,   
    "NUM_EXP" : 10,  
    "NUM_TASK" : 4,  
    }
elif SETTING == '0':
    HPARAM = {
    "batch_size": 128,        
    "num_epoch": 10,           
    "start_lr": 0.01,
    "alpha": 1,   
    "temperature": 2,   
    "NUM_EXP" : 50,  
    "NUM_TASK" : 9,  
    }

In [4]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomCNN, self).__init__()

        # Adjusting the first Conv2d to take 512 channels as input (from ResNet-18 output)
        self.features = nn.Sequential(
            # First convolution: 512 input channels to 128 output channels (reduce parameters)
            nn.Conv2d(512, 128, kernel_size=1, stride=1, padding=0),  # 1x1 kernel reduces parameters
            nn.ReLU(inplace=True),
            # Second convolution: 128 input channels to 64 output channels (further reduce)
            nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0),  # 1x1 kernel reduces parameters
            nn.ReLU(inplace=True),
            # Ensure fixed spatial size of 1x1 using AdaptiveAvgPool2d
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Dropout(p=0.25),
        )

        # Classifier will take the 64 channels (final output from features)
        self.classifier = nn.Sequential(
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the output for classifier
        x = self.classifier(x)
        return x



class MTCustomCNN(CustomCNN, MultiTaskModule):
    """
    Convolutional Neural Network
    with multi-head classifier
    """

    def __init__(self):
        super().__init__()
        self.classifier = MultiHeadClassifier(64)


    def forward(self, x, task_labels):

        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the output for classifier
        x = self.classifier(x, task_labels)
        return x

class ResNetCNN(MultiTaskModule):

    def __init__(self):

        super().__init__()
        self.resnet18 = models.resnet18(pretrained=True)

        self.resnet_backbone = nn.Sequential(
            self.resnet18.conv1,    # Initial conv layer
            self.resnet18.bn1,      # Batch normalization
            self.resnet18.relu,     # Activation
            self.resnet18.maxpool,  # Max pooling
            self.resnet18.layer1,   # Layer 1
            self.resnet18.layer2,   # Layer 2
            self.resnet18.layer3,    # Layer 3 (final output layer)
            self.resnet18.layer4
)
        self.classifier = MTCustomCNN()

        for param in self.resnet_backbone.parameters():
            param.requires_grad = False

    def forward_single_task(self, x, task_id):
        x = self.resnet_backbone(x)  # Passing through the initial layers of ResNet-18
        out = self.classifier(x, task_id)
        return out

In [5]:
class_order = [73, 48, 17, 32, 11, 62, 68, 92, 91, 3, 77, 79, 43, 88, 47, 82, 13, 78, 70, 90, 12, 37, 2, 76, 84, 98, 59, 96, 52, 93, 26, 45, 20, 46, 29, 56, 97, 44, 35, 58, 5, 8, 94, 54, 67, 27, 99, 1, 25, 42, 0, 4, 6, 7, 9, 10, 14, 15, 16, 18, 19, 21, 22, 23, 24, 28, 30, 31, 33, 34, 36, 38, 39, 40, 41, 49, 50, 51, 53, 55, 57, 60, 61, 63, 64, 65, 66, 69, 71, 72, 74, 75, 80, 81, 83, 85, 86, 87, 89, 95]

In [6]:
import torchvision.transforms as transforms

# print to stdout
interactive_logger = InteractiveLogger()

normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
train_transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)
test_transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)

benchmark = SplitCIFAR100(
    n_experiences = HPARAM["NUM_EXP"],          
    return_task_id = True,
    fixed_class_order= class_order, 
    train_transform = train_transform,
    eval_transform = test_transform,
    
)

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(
        num_classes=NUM_CLASSES[DATASET_NAME], save_image=False, stream=True
    ),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=interactive_logger,
)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
MODEL_NAME = 'ResNetCNN'
RUN = '0'                    #Multiple runs 0,1,2
model = ResNetCNN()

optimizer = Adam(model.parameters(), HPARAM["start_lr"])

cl_strategy = LwF(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    train_mb_size=HPARAM["batch_size"],
    train_epochs=HPARAM["num_epoch"],
    eval_mb_size=HPARAM["batch_size"],
    alpha=HPARAM["alpha"],              # LwF parameter
    temperature=HPARAM["temperature"],  # LwF parameter
    evaluator=eval_plugin,
    device=device,
)

if SAVE:
    DATA_ROOT = ROOT / DATASET_NAME / MODEL_NAME / RUN
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

/home/uregina/env_name/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/uregina/env_name/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
print("Starting experiment...")
results_dict = {}  # Use a dictionary instead of a list
for index, experience in enumerate(benchmark.train_stream):
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    res = cl_strategy.train(experience)
    print("Training completed")
    print("Computing accuracy on the whole test")
    results_dict[index] = cl_strategy.eval(benchmark.test_stream)  # Use the index as the key
    if index == HPARAM["NUM_TASK"]:
        break

print("Experiment completed")

Starting experiment...
Start of experience:  0
Current Classes:  [32, 3, 68, 73, 11, 48, 17, 91, 92, 62]
-- >> Start of training phase << --
100%|██████████| 40/40 [00:12<00:00,  3.16it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 478.4258
	DiskUsage_MB/train_phase/train_stream/Task000 = 478.4258
	Loss_Epoch/train_phase/train_stream/Task000 = 1.3797
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0218
	Time_Epoch/train_phase/train_stream/Task000 = 12.6801
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4930
 25%|██▌       | 10/40 [00:03<00:10,  2.80it/s]

KeyboardInterrupt: 

In [ ]:
if SAVE:
    file_name = f"{MODEL_NAME}_{DATASET_NAME}_{RUN}_results.txt"
    file_path = ROOT / DATASET_NAME / MODEL_NAME / RUN / file_name
    with open(file_path, "w") as file:
        file.write(f"Model: {MODEL_NAME}\n")
        file.write(f"Dataset: {DATASET_NAME}\n")
        file.write(f"Run: {RUN}\n")
        file.write(f"Setting: {SETTING}\n")    
        file.write("\nResults Dictionary:\n")
        file.write("--------------------------------------------------\n")
        for key, value in results_dict.items():
            file.write(f"Experience {key}:\n")
            for metric, metric_value in value.items():
                # Convert tensors to lists for saving
                if isinstance(metric_value, torch.Tensor):
                    metric_value = metric_value.tolist()
                file.write(f"  {metric}: {metric_value}\n")
            file.write("--------------------------------------------------\n")